# Hybrid Classical Quantum AutoEncoder for anomaly detection

## VQC
The VQC implemented is the number 10 of the reference paper, which essentially is a stack Pauli $YX$ rotation gate and a circular series of controlled $CX$, stacked while alternated with an encoding based on the Pauli $X$ rotation

In [8]:
from qiskit.circuit import QuantumCircuit, Gate, ParameterVector
from qiskit.opflow.expectations import PauliExpectation
from qiskit_machine_learning.connectors import TorchConnector

def get_encoding_block(nqubits: int, features: ParameterVector) -> Gate:
    """ n parameters required """
    assert len(features) == nqubits 
    block = QuantumCircuit(nqubits, name="Encoding Block")
    for i in range(nqubits):
        block.rx(features[i], i)
    return block.to_gate()

def get_ansatz_block(nqubits: int, parameters: ParameterVector) -> Gate:
    """
    need 2n parameters
    """
    assert nqubits * 2 == len(parameters)
    block = QuantumCircuit(nqubits, name="Ansatz Block")
    for i in range(nqubits):
        block.ry(parameters[i], i)
        block.rx(parameters[i + nqubits], i)
    if nqubits > 1:
        block.cx(nqubits - 1, 0)
        for i in range(nqubits - 1):
            block.cx(i, i + 1)
    return block.to_gate()

def get_ansatz(nqubits: int, parameters: ParameterVector, features: ParameterVector, reps: int=3) -> QuantumCircuit:
    assert len(parameters) == reps * 2 * nqubits
    ansatz = QuantumCircuit(nqubits)
    ansatz.compose(get_ansatz_block(nqubits, parameters[:2 * nqubits]), range(nqubits), inplace=True)
    for i in range(1, reps):
        ansatz.barrier()
        ansatz.compose(get_encoding_block(nqubits, features), range(nqubits), inplace=True)
        ansatz.barrier()
        ansatz.compose(get_ansatz_block(nqubits, parameters[2 * nqubits * i :2 * nqubits * (i + 1)]), range(nqubits), inplace=True)
    return ansatz

def get_ansatz_ws(nqubits: int, parameters: ParameterVector, features: ParameterVector, reps: int=3) -> QuantumCircuit:
    assert len(parameters) == 2 * nqubits
    ansatz = QuantumCircuit(nqubits)
    ansatz.compose(get_ansatz_block(nqubits, parameters), range(nqubits), inplace=True)
    for i in range(1, reps):
        ansatz.barrier()
        ansatz.compose(get_encoding_block(nqubits, features), range(nqubits), inplace=True)
        ansatz.barrier()
        ansatz.compose(get_ansatz_block(nqubits, parameters), range(nqubits), inplace=True)
    return ansatz


In [19]:
from qiskit.utils import algorithm_globals
algorithm_globals.random_seed = 528491

size = (10, 4)
data = algorithm_globals.random.random(size)
nqubits = data.shape[1]
print(data, nqubits)

[[0.80725468 0.26761882 0.94861625 0.50421627]
 [0.7698305  0.2292505  0.9626732  0.8040316 ]
 [0.85501103 0.85395559 0.69492289 0.49207791]
 [0.49374971 0.50729469 0.60206583 0.75959243]
 [0.18578251 0.68047336 0.7272356  0.26538521]
 [0.15946585 0.58420932 0.95539176 0.30436282]
 [0.52903154 0.47291539 0.33246663 0.76808705]
 [0.66603794 0.78136749 0.50693698 0.39778749]
 [0.83293417 0.77548085 0.08815    0.57324428]
 [0.23076437 0.94343524 0.12147385 0.37204892]] 4


In [20]:
input = ParameterVector("x", data.shape[1])
weights = ParameterVector("theta", nqubits * 2)
circuit = get_ansatz_ws(nqubits, weights, input)
circuit.draw()


»
q_0: »
     »
q_1: »
     »
q_2: »
     »
q_3: »
     »
«     ┌────────────────────────────────────────────────────────────────────────────────────────┐»
«q_0: ┤0                                                                                       ├»
«     │                                                                                        │»
«q_1: ┤1                                                                                       ├»
«     │  Ansatz Block(theta[0],theta[1],theta[2],theta[3],theta[4],theta[5],theta[6],theta[7]) │»
«q_2: ┤2                                                                                       ├»
«     │                                                                                        │»
«q_3: ┤3                                                                                       ├»
«     └────────────────────────────────────────────────────────────────────────────────────────┘»
«      ░ ┌──────────────────────────────────────┐ ░ »
«q_0: ─░─┤0                                     ├─░─»
«      ░ │                                      │ ░ »
«q_1: ─░─┤1                                     ├─░─»
«      ░ │  Encoding Block(x[0],x[1],x[2],x[3]) │ ░ »
«q_2: ─░─┤2                                     ├─░─»
«      ░ │                                      │ ░ »
«q_3: ─░─┤3                                     ├─░─»
«      ░ └──────────────────────────────────────┘ ░ »
«     ┌────────────────────────────────────────────────────────────────────────────────────────┐»
«q_0: ┤0                                                                                       ├»
«     │                                                                                        │»
«q_1: ┤1                                                                                       ├»
«     │  Ansatz Block(theta[0],theta[1],theta[2],theta[3],theta[4],theta[5],theta[6],theta[7]) │»
«q_2: ┤2                                                                                       ├»
«     │                                                                                        │»
«q_3: ┤3                                                                                       ├»
«     └────────────────────────────────────────────────────────────────────────────────────────┘»
«      ░ ┌──────────────────────────────────────┐ ░ »
«q_0: ─░─┤0                                     ├─░─»
«      ░ │                                      │ ░ »
«q_1: ─░─┤1                                     ├─░─»
«      ░ │  Encoding Block(x[0],x[1],x[2],x[3]) │ ░ »
«q_2: ─░─┤2                                     ├─░─»
«      ░ │                                      │ ░ »
«q_3: ─░─┤3                                     ├─░─»
«      ░ └──────────────────────────────────────┘ ░ »
«     ┌────────────────────────────────────────────────────────────────────────────────────────┐
«q_0: ┤0                                                                                       ├
«     │                                                                                        │
«q_1: ┤1                                                                                       ├
«     │  Ansatz Block(theta[0],theta[1],theta[2],theta[3],theta[4],theta[5],theta[6],theta[7]) │
«q_2: ┤2                                                                                       ├
«     │                                                                                        │
«q_3: ┤3                                                                                       ├
«     └────────────────────────────────────────────────────────────────────────────────────────┘

In [21]:
random_weights = algorithm_globals.random.random(2 * nqubits)
print(random_weights)

[0.31544562 0.04572466 0.8256722  0.25411594 0.53149553 0.23323015
 0.59966905 0.13732588]


In [ ]:
from qiskit_machine_learning.neural_networks import SamplerQNN

sqnn_ws = SamplerQNN(
    circuit=circuit,
    input_params=input,
    weight_params=weights,
    interpret=lambda x: "{:b}".format(x).count('1') % 2 == 0, # parity check
    output_shape=2
)
print(sqnn_ws)

In [ ]:
sampler_qnn_forward = sqnn_ws.forward(data[0], random_weights) # require encoding + ansatz parameters, result is a ndarray
print(f"Forward pass result for SamplerQNN: {sampler_qnn_forward}. \nShape: {sampler_qnn_forward.shape}")


Forward pass result for SamplerQNN: [[0.43589833 0.43589833]]. 
Shape: (1, 2)


In [ ]:
input1 = ParameterVector("x1", data.shape[1])
weights1 = ParameterVector("theta1", nqubits * 2 * 3)
circuit = get_ansatz(nqubits, weights, input)
circuit.draw()
